In [21]:
!pip install accelerate

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig,AutoModelForCausalLM
import pandas as pd
import random
from tqdm import tqdm

In [1]:
import torch

In [2]:
torch.cuda.device_count()

1

In [127]:
model_name = 'neulab/omnitab-large'

In [3]:
def load_model(model_name):
    config = AutoConfig.from_pretrained(model_name)
    # IMPORTANT: the initial BART model's decoding is penalized by no_repeat_ngram_size, and thus
    # we should disable it here to avoid problematic generation
    config.no_repeat_ngram_size = 3 # В omnitab = 3, part exec = 0
    config.max_length = 1024
    #config.early_stopping = False #  В омнитаб нет

    #model = AutoModel.from_config(config) # Для создания экземпляра модели без предобученных весов
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return tokenizer, model

In [129]:
tokenizer, model = load_model(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [131]:
next(model.parameters()).device

device(type='cuda', index=0)

In [4]:
from datasets import load_dataset
import json

In [22]:
data_files = {"train":"data/wtq_lf_preorder/train/data-00000-of-00001.arrow",
              "test":"data/wtq_lf_preorder/test/data-00000-of-00001.arrow",
              "validation":"data/wtq_lf_preorder/validation/data-00000-of-00001.arrow"}
datasets = load_dataset("arrow", data_files=data_files)
#datasets = load_dataset("Stanford/wikitablequestions")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 8610
    })
    test: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 4344
    })
    validation: Dataset({
        features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
        num_rows: 2092
    })
})

In [23]:
datasets = datasets["test"]
datasets

Dataset({
    features: ['id', 'question', 'answers', 'table', 'lf_p', 'lf_pc', 'lf_pcs', 'lf_pcsgbh', 'lf_pcsgbhob', 'lf_pcsgbhoba', 'lf_pcsgbhobaop', 'lf_pcsgbhobaopl', 'sql'],
    num_rows: 4344
})

In [88]:
#features = ["question","table"]
#def length_filter(example):
#    return {f"length_{target_col_name}": len(tokenizer.tokenize(example[target_col_name]))<=max_target_length}
#
#for mode in ['train',"validation"]:#
#
#   if mode =="train":
#        datasets[mode] = datasets[mode].map(length_filter)
#        filter_col = [idx for idx, i in enumerate(datasets[mode][filter_col_name]) if i==1]
#        logger.info(f"remove {len(datasets[mode])-len(filter_col)} observations with length > {max_target_length}.")
#        logger.info(f"Previous {mode} size {len(datasets[mode])} actual {mode} size {len(filter_col)}")
#        datasets[mode] = datasets[mode].select(filter_col)
#
#    removed_cols = set(datasets[mode].features.keys())-set([target_col_name]+features)
#    datasets[mode] = datasets[mode].remove_columns(removed_cols)
#    datasets[mode] = datasets[mode].rename_column(target_col_name, 'answers')

In [5]:
from data_processing.metrics import (to_value_list,
                                     to_value,
                                     plot_and_save_model_performance,
                                     fuzzy_matching,
                                     strict_denotation_accuracy,
                                     flexible_denotation_accuracy)

/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<ipykernel.iostream.OutStream object at 0x7fd2c74c9460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-seen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7fd2c74c9460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-unseen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7fd2c74c9460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/training.tagged


In [109]:
path_model_folder = "./models/OmniTab"
output_file = f'{path_model_folder}/omnitab_large_evaluation_results.json'
#target_col_name = f"lf_{Omega_include.lower()}"
#Omega_include = path_model_folder.split('/')[-1].split('_')[1]

In [ ]:
#removed_cols = set(datasets["validation"].features.keys())-set([target_col_name]+features+["answers"])
#datasets["validation"] = datasets["validation"].remove_columns(removed_cols)
#datasets["validation"] = datasets["validation"].rename_column(target_col_name, 'answers_lf')
#datasets["validation"] = datasets["validation"].rename_column("answers", 'answers_exec')
#datasets["test"] = datasets["test"].rename_column("answers", 'answers_exec')

In [6]:
rnd = random.randint(0, len(datasets))

NameError: name 'datasets' is not defined

In [92]:
rnd

2986

In [10]:
def preprocess_tableqa_function(example, is_test=False):

        question = example["question"].lower()
        example_table = example["table"]
        table = pd.DataFrame.from_records(example_table["rows"], columns=example_table["header"])
        
        model_input = tokenizer(
            table=table, query=question, max_length=1024, padding=False, truncation=True
        )
            
        return model_input


In [11]:
def evaluate_model(model, dataset, comp_graph = False, is_test=False,flatten_mode="preorder"):
        model.eval()
        perfs={ "Strict_Denotation_Accuracy_Execs":[]}
        for idx, example in tqdm(enumerate(dataset), total=len(dataset)):
            proces_example = preprocess_tableqa_function(example)
            input_ids = proces_example["input_ids"]
            attention_mask = proces_example["attention_mask"]

            input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
            attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            prediction_lf = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True).strip()
            if comp_graph:
                header=tokenizer.decode(input_ids.squeeze(), skip_special_tokens=True)
                header=header.split(' col : ')[1].split(' row 1 : ')[0].split(' |')
                header=[h.strip() for h in header]
                #G=parse(prediction_lf, header, flatten_mode=args.flatten_mode)
                try:
                    print('1')
                    G=parse(prediction_lf, header, flatten_mode=flatten_mode)
                    print('2')
                    prediction_exec = G.executed_last_node()
                    print(f'answers_exec {answers_exec} prediction_exec {prediction_exec}')
                    acc_ex_stric = strict_denotation_accuracy(to_value_list(answers_exec), to_value_list(prediction_exec))

                except Exception as e:
                    print(e)
                    acc_ex_stric = False

            
            else:
                prediction_exec = prediction_lf
                answers_exec = example['answers']
                print(prediction_exec,answers_exec)
                acc_ex_stric = strict_denotation_accuracy(to_value_list(answers_exec), to_value_list([prediction_exec]))
            print(f'Strict_Denotation_Accuracy_Execs {acc_ex_stric}')
            perfs["Strict_Denotation_Accuracy_Execs"].append(acc_ex_stric)

        Strict_Denotation_Accuracy_Exec = sum(perfs["Strict_Denotation_Accuracy_Execs"])/len(perfs["Strict_Denotation_Accuracy_Execs"])

        return {'Strict_Denotation_Accuracy_Exec': Strict_Denotation_Accuracy_Exec}  # example


In [113]:
out = evaluate_model(model,datasets)
with open(output_file,"w") as out_f:
    json.dump(out,out_f)

  0%|          | 1/4344 [00:00<28:11,  2.57it/s]

1 ['Italy']
Strict_Denotation_Accuracy_Execs False


  0%|          | 2/4344 [00:00<26:23,  2.74it/s]

100,000 ['100,000']
Strict_Denotation_Accuracy_Execs True


  0%|          | 3/4344 [00:01<33:14,  2.18it/s]

...

bill glasson, jay haas, neal lancaster, neall lancastry, davis love iii, jeff maggert, phil mickelson, bob tway ['Bob Tway']
Strict_Denotation_Accuracy_Execs False


100%|█████████▉| 4343/4344 [37:42<00:00,  1.44it/s]

43.12 ['43.12']
Strict_Denotation_Accuracy_Execs True


100%|██████████| 4344/4344 [37:42<00:00,  1.92it/s]

0 ['6']
Strict_Denotation_Accuracy_Execs False


In [96]:
out

{'Strict_Denotation_Accuracy_Exec': 0.6180939226519337}

In [67]:
preprocess_tableqa_function(datasets[0])


{'input_ids': [0, 5488, 247, 56, 5, 144, 19294, 2073, 624, 5, 299, 158, 116, 11311, 4832, 17816, 1721, 25544, 661, 1721, 2711, 1721, 3421, 1721, 7437, 100, 1698, 31814, 37457, 282, 46340, 3236, 112, 4832, 112, 1721, 8445, 25787, 3767, 2802, 2794, 50141, 1640, 1723, 510, 43, 1721, 8316, 8035, 385, 108, 39318, 5384, 858, 1721, 195, 298, 1132, 108, 158, 113, 1721, 843, 3236, 132, 4832, 132, 1721, 21441, 338, 16052, 2413, 858, 705, 50141, 1640, 500, 3048, 43, 1721, 2711, 230, 3632, 22506, 139, 788, 1721, 579, 4, 90, 4, 1721, 389, 3236, 155, 4832, 155, 1721, 7234, 1949, 47045, 179, 50141, 1640, 2068, 250, 43, 1721, 7965, 1168, 7864, 254, 1721, 579, 4, 90, 4, 1721, 564, 3236, 204, 4832, 204, 1721, 5476, 5675, 23479, 2531, 50141, 1640, 2068, 250, 43, 1721, 13287, 10577, 1721, 579, 4, 90, 4, 1721, 291, 3236, 195, 4832, 195, 1721, 14185, 29755, 1210, 14683, 50141, 1640, 2068, 250, 43, 1721, 26185, 1023, 281, 1721, 579, 4, 90, 4, 1721, 379, 3236, 231, 4832, 231, 1721, 16439, 4011, 611, 1417, 501

In [72]:
datasets[0]["answers"]

['Italy']

In [117]:
import json
import pandas as pd

In [118]:
with open('models/OmniGraph/model_evaluation_results.json','r') as inf:
    sda = pd.DataFrame(json.load(inf))

In [122]:
sda.T[sda.T['Strict_Denotation_Accuracy_Exec'] == sda.T['Strict_Denotation_Accuracy_Exec'].max()]

,Denotation_Accuracy,Fuzzy_Match,Strict_Denotation_Accuracy_Exec,Flexible_Denotation_Accuracy_Exec
models/OmniGraph/checkpoint-4000,0.495698,71.472275,0.672084,0.685946


In [123]:
with open('models/PartialExec/model_evaluation_results.json','r') as inf:
    sda = pd.DataFrame(json.load(inf))

# Tapex-large

In [24]:
path_model_folder = "./models/Tapex"
output_file = f'{path_model_folder}/tapex_large_evaluation_results.json'

In [25]:
model_name = 'microsoft/tapex-large'

In [26]:
tokenizer, model = load_model(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [27]:
out = evaluate_model(model,datasets)
with open(output_file,"w") as out_f:
    json.dump(out,out_f)

  0%|          | 2/4344 [00:00<09:05,  7.96it/s]

 ['Italy']
Strict_Denotation_Accuracy_Execs False
 ['100,000']
Strict_Denotation_Accuracy_Execs False


  0%|          | 4/4344 [00:00<09:55,  7.29it/s]

 ['17 years']
Strict_Denotation_Accuracy_Execs False
 ['January 26, 1995']
Strict_Denotation_Accuracy_Execs False


  0%|          | 6/4344 [00:00<09:48,  7.37it/s]

...

 ['5532']
Strict_Denotation_Accuracy_Execs False
 ['5']
Strict_Denotation_Accuracy_Execs False


100%|█████████▉| 4343/4344 [10:23<00:00,  7.28it/s]

 ['Bob Tway']
Strict_Denotation_Accuracy_Execs False
 ['43.12']
Strict_Denotation_Accuracy_Execs False


100%|██████████| 4344/4344 [10:24<00:00,  6.96it/s]

 ['6']
Strict_Denotation_Accuracy_Execs False


# Tapex-large fituning on WTQ

In [18]:
path_model_folder = "./models/Tapex"
output_file = f'{path_model_folder}/tapex_large_finetuned_wtq_evaluation_results.json'

In [19]:
model_name = 'microsoft/tapex-large-finetuned-wtq'

In [20]:
tokenizer, model = load_model(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

config.json:   0%|          | 0.00/951 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/246 [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [21]:
out = evaluate_model(model,datasets)
with open(output_file,"w") as out_f:
    json.dump(out,out_f)

  0%|          | 3/4344 [00:00<02:45, 26.17it/s]

ita ['Italy']
Strict_Denotation_Accuracy_Execs False
100,000 ['100,000']
Strict_Denotation_Accuracy_Execs True
6 years ['17 years']
Strict_Denotation_Accuracy_Execs False
january 26, 1995 ['January 26, 1995']
Strict_Denotation_Accuracy_Execs True
16 ['17']
Strict_Denotation_Accuracy_Execs False


  0%|          | 9/4344 [00:00<03:07, 23.15it/s]

world junior championships ['World Junior Championships']
Strict_Denotation_Accuracy_Execs True
15 ['15']
Strict_Denotation_Accuracy_Execs True
363 ['363']
Strict_Denotation_Accuracy_Execs True
1982-1985 ['1982-1985']
Strict_Denotation_Accuracy_Execs True
2002 ['2000']
Strict_Denotation_Accuracy_Execs False


  0%|          | 15/4344 [00:00<03:38, 19.78it/s]

...

dionisio rodríguez, "dioni" ['Regina Do Santos']
Strict_Denotation_Accuracy_Execs False
16 ['1']
Strict_Denotation_Accuracy_Execs False
6 ['5']
Strict_Denotation_Accuracy_Execs False
258 ['16']
Strict_Denotation_Accuracy_Execs False
1 ['5']
Strict_Denotation_Accuracy_Execs False
11 ['10']
Strict_Denotation_Accuracy_Execs False


100%|█████████▉| 4339/4344 [03:51<00:00, 20.74it/s]

colosimo ['Colosimo']
Strict_Denotation_Accuracy_Execs True
alpine ['Plumas']
Strict_Denotation_Accuracy_Execs False
yes ['no']
Strict_Denotation_Accuracy_Execs False
2267 ['5532']
Strict_Denotation_Accuracy_Execs False
5 ['5']
Strict_Denotation_Accuracy_Execs True


100%|██████████| 4344/4344 [03:51<00:00, 18.76it/s]


bob tway ['Bob Tway']
Strict_Denotation_Accuracy_Execs True
43.12 ['43.12']
Strict_Denotation_Accuracy_Execs True
14 ['6']
Strict_Denotation_Accuracy_Execs False


# BART - large

In [1]:
from data_processing.seq_to_graph import parse
from data_processing.nodes import create_nodes
from data_processing.metrics import target_values_map, flexible_denotation_accuracy, to_value_list

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
/opt/conda/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


<ipykernel.iostream.OutStream object at 0x7fa86fa5d460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-seen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7fa86fa5d460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/pristine-unseen-tables.tagged
<ipykernel.iostream.OutStream object at 0x7fa86fa5d460> Reading dataset from /home/jovyan/cloud/postgraduate/works/Partial-Exec/data/tables/tagged/training.tagged


In [19]:
from data_processing.utils import find_first_edges

def find_layer_neighboors(parent_node_names,edges):
    layer_neighboors = []
    for node in parent_node_names:
        layer_neighboors.extend(find_neighboors(node,edges))
    return set(layer_neighboors)

def find_neighboors(node_name, edges):
    return set(edge[1] for edge in list(filter(lambda t: t[0] == node_name, edges)))

def sort_graphe_execute_nodes(edges):
    sorted_nodes = []
    sorted_nodes.extend(find_first_edges(edges))
    layer_neighboors = sorted_nodes
    
    while len(layer_neighboors) > 0:
        print(layer_neighboors)
        layer_neighboors = find_layer_neighboors(layer_neighboors,edges)
        sorted_nodes.extend(layer_neighboors)
    return sorted_nodes


def translate_query_to_graph_form(query,answer=None,flatten_mode = 'preorder',
                                  Omega_include=["P","C","S","GB","H","OB","A","OP","L"],task='tapex'):
    pattern = ' col : '
    try:
        sql,_ = query.split(pattern)
    except Exception as e:
        print(e)
    sql  = sql.strip()
    df = deserializ_tapex_linear_table(" col : "+query.split(" col : ")[1])
    
    new_column_names = []
    for head in df.columns:
        head_new = "_".join(head.split(' '))
        sql = re.sub(head,f'"{head_new}"',sql)
        new_column_names.append(f'"{head_new}"')
        
    df.columns = new_column_names
    df['agg'] = np.zeros(df.shape[0])
    edges = []
    condi_expressions = {}
    print(sql)
    G0 = create_nodes(sql, df=df)
    print('ok')
    if task == 'tapex':
        #sort_nodes = sort_graphe_execute_nodes(edges)
        result = to_value_list(G0.executed_last_node())
        #sort_nodes = sort_graphe_execute_nodes(edges)
        #transformed_query = ' NODE '+f' NODE '.join(sort_nodes)
        #return transformed_query+serialize_table_to_tapex_format(df), result
    
import json

import re
import pandas as pd
def deserializ_tapex_linear_table(linear_table):
    
    head_pattern = r" col :(.*) row 1 : "
    row_pattern = r" row \d+ : "
    coll_delimetr = " | "
    heads = [h.strip() for h in 
             re.search(head_pattern,linear_table)[0].replace(" col : ","",1).replace(" row 1 : ","",1).split(coll_delimetr)]
    rows = [[r.strip() for r in row.split(coll_delimetr)] for row in re.split(row_pattern,linear_table)[1:]]
    return pd.DataFrame(rows,columns=heads)

def serialize_table_to_tapex_format(df):
    head_pattern = " col : "
    row_pattern = " row {num} : "
    coll_delimetr = " | "
    
    lin_table = head_pattern+coll_delimetr.join(df.columns)
    for i,row in df.iterrows():
        print(row_pattern.format(num=i+1))
        lin_table+=row_pattern.format(num=i+1)+coll_delimetr.join(str(r) for r in row.values)
    
    return lin_table

In [20]:
a,b = translate_query_to_graph_form("select san. where car = 'ford escort rs2000' col : position | driver | no. | car | entrant | lak. | ora. | san. | phi. | total row 1 : 1 | john briggs | 9 | honda prelude, chevrolet | john briggs | 42 | 42 | 42 | 42 | 168 row 2 : 2 | kerry baily | 18 | toyota supra, chevrolet | kerry baily | 38 | 38 | 36 | 38 | 150 row 3 : 3 | james phillip | 55 | honda prelude, chevrolet | james phillip | 26 | 28 | 28 | 30 | 112 row 4 : 4 | mick monterosso | 2 | ford escort rs2000 | mick monterosso | none | 34 | 36 | 34 | 104 row 5 : 5 | bob jolly | 3 | holden vs commodore | bob jolly | none | 28 | 16 | 32 | 76 row 6 : 6 | danny osborne | none | mazda rx-7 | none | 26 | 10 | 30 | none | 66 row 7 : 7 | mike imrie | 4 | saab | imrie motor sport | 23 | 11 | none | 28 | 62 row 8 : 8 | mark trenoweth | none | jaguar | none | 33 | 24 | none | none | 57 row 9 : 9 | ivan mikac | 42 | mazda rx-7 | ivan mikac | none | none | 25 | 26 | 51 row 10 : 10 | des wall | none | toyota supra | none | 15 | 32 | none | none | 47 row 11 : 11 | kevin clark | 116 | ford mustang gt | kevin clark | none | none | 23 | 23 | 46 row 12 : 12 | peter o'brien | 17 | holden vl commodore | o'brien aluminium | none | 11 | 29 | none | 40 row 13 : 13 | chris fing | none | chevrolet monza | none | 29 | none | none | none | 29 row 14 : 14 | brian smith | none | alfa romeo gtv, chevrolet | none | none | 28 | none | none | 28 row 15 : 15 | gary rowe | 47 | nissan stanza | gary rowe | none | none | 21 | none | 21 row 16 : 15 | domenic beninca | none | none | none | none | none | none | 21 | 21 row 17 : 17 | phil crompton | 49 | ford ea falcon | phil crompton | 17 | none | none | none | 17 row 18 : 18 | allan mccarthy | none | alfa romeo alfetta | none | 14 | none | none | none | 14 row 19 : 19 | chris donnelly | none | none | none | 12 | none | none | none | 12 row 20 : 19 | paul barrett | none | none | none | none | none | none | 12 | 12 row 21 : 21 | brett francis | none | none | none | 11 | none | none | none | 11 row 22 : 22 | shane eklund | none | none | none | 10 | none | none | none | 10 row 23 : 22 | craig wildridge | none | none | none | none | 10 | none | none | 10 row 24 : 22 | ron o'brien | none | none | none | none | none | none | 10 | 10")

select "san." where "car" = 'ford escort rs2000'
[(SELECT expressions: 
  (COLUMN this: 
    (IDENTIFIER this: san., quoted: True)), where: 
  (WHERE this: 
    (EQ this: 
      (COLUMN this: 
        (IDENTIFIER this: car, quoted: True)), expression: 
      (LITERAL this: ford escort rs2000, is_string: True))))]
sub-query SELECT "san." WHERE "car" = 'ford escort rs2000' type <class 'sqlglot.expressions.Select'>
ok


TypeError: cannot unpack non-iterable NoneType object

In [5]:
import numpy as np